# Does a certain area (location) have higher/lower adoption rates?

In [2]:
import pandas as pd
import numpy as np
import requests
import petpy
import pprint

from matplotlib import pyplot as plt
from config import api_key as api_key
from config import api_secret as api_secret

## Retrive data from APIs

**API/Resources:**
- [Adopt a Pet API](https://www.adoptapet.com/public/apis/pet_list.html)
- [Pet Finder API](https://www.petfinder.com/developers/)


In [24]:
pf = petpy.Petfinder(key=api_key, secret=api_secret)

Adoptable dogs (after Jan 1st, 2018): 18
Found dogs (after Jan 1st, 2018): 100
Adopted dogs (after Jan 1st, 2018): 100


In [30]:
adoptable_dogs_2018 = pf.animals(animal_type = 'dog', status = 'adoptable', location = 'Charlotte, NC', sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
found_dogs_2018 = pf.animals(animal_type = 'dog', status = 'found', location = 'Charlotte, NC', sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)



adoptable_found_dogs = pd.merge(adoptable_dogs_2018, found_dogs_2018, on='type')
adoptable_found_dogs

,id_x,organization_id_x,url_x,type,species_x,age_x,gender_x,size_x,coat_x,tags_x,...,contact.phone_y,contact.address.address1_y,contact.address.address2_y,contact.address.city_y,contact.address.state_y,contact.address.postcode_y,contact.address.country_y,animal_id_y,animal_type_y,organization_id_y
0,43133069,SC125,https://www.petfinder.com/dog/gilley-43133069/...,Dog,Dog,Adult,Female,Medium,Short,"[Energetic, Friendly, Loving, Loves toys, Affe...",...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
1,43072632,NC399,https://www.petfinder.com/dog/billy-bob-430726...,Dog,Dog,Young,Male,Medium,None,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
2,42909210,NC91,https://www.petfinder.com/dog/james-42909210/n...,Dog,Dog,Adult,Male,Large,None,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
3,42732523,NC626,https://www.petfinder.com/dog/king-42732523/nc...,Dog,Dog,Adult,Male,Medium,Short,"[, Athletic, Loyal, Affectionate]",...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
4,42363798,NC421,https://www.petfinder.com/dog/alfonso-42363798...,Dog,Dog,Young,Male,Medium,Short,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
5,41897500,TN488,https://www.petfinder.com/dog/eric-41897500/nc...,Dog,Dog,Adult,Male,Medium,Short,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
6,41766948,NC91,https://www.petfinder.com/dog/lola-41766948/nc...,Dog,Dog,Adult,Female,Large,None,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
7,41736687,NC968,https://www.petfinder.com/dog/emmie-foster-hom...,Dog,Dog,Adult,Female,Large,Short,"[Loving, playful, active, smart, Friendly, ...",...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
8,41686835,NC834,https://www.petfinder.com/dog/lulu-41686835/nc...,Dog,Dog,Adult,Female,Large,None,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527
9,41586546,NC626,https://www.petfinder.com/dog/brantley-4158654...,Dog,Dog,Adult,Male,Medium,Short,[],...,None,P. O. Box 204,None,Barium Springs,NC,28010,US,41979725,dog,nc527


In [27]:
#2018 Dog + Cat Information
adoptable_dogs_2018 = pf.animals(animal_type = 'dog', status = 'adoptable', location = 'Charlotte, NC', sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
print('Adoptable dogs (2018): '+ str(len(adoptable_dogs['id'].unique())))

found_dogs_2018 = pf.animals(animal_type = 'dog', status = 'found', location = 'Charlotte, NC', sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
print('Found dogs (2018): '+ str(len(found_dogs['id'].unique())))

adopted_dogs_2018 = pf.animals(animal_type = 'dog', status = 'adopted', location = 'Charlotte, NC', pages=10, sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
print('Adopted dogs (2018): '+ str(len(adopted_dogs['id'].unique())))

#adoptable_found_dogs = pd.merge(adoptable_dogs_2018, found_dogs_2018, on='location')
print(found_dogs_2018.head())

adoptable_cats_2018 = pf.animals(animal_type = 'cat', status = 'adoptable', location = 'Charlotte, NC', sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
print('Adoptable cats (2018): '+ str(len(adoptable_dogs['id'].unique())))

found_cats_2018 = pf.animals(animal_type = 'cat', status = 'found', location = 'Charlotte, NC', sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
print('Found cats (2018): '+ str(len(found_dogs['id'].unique())))

adopted_cats_2018 = pf.animals(animal_type = 'cat', status = 'adopted', location = 'Charlotte, NC', pages=10, sort='recent', after_date='2018-01-01', before_date = '2019-01-01', results_per_page = 100, return_df=True)
print('Adopted cats (2018): '+ str(len(adopted_dogs['id'].unique())))


Adoptable dogs (2018): 18
Found dogs (2018): 100
Adopted dogs (2018): 100
         id  organization_id  \
0  41979725            NC527   

                                                 url type species    age  \
0  https://www.petfinder.com/dog/micah-41979725/n...  Dog     Dog  Adult   

   gender   size  coat tags  ... contact.phone contact.address.address1  \
0  Female  Small  Long   []  ...          None            P. O. Box 204   

  contact.address.address2 contact.address.city contact.address.state  \
0                     None       Barium Springs                    NC   

  contact.address.postcode contact.address.country animal_id animal_type  \
0                    28010                      US  41979725         dog   

   organization_id  
0            nc527  

[1 rows x 46 columns]
Adoptable cats (2018): 18


KeyError: '_links.organization.href'

In [43]:


#Data Frame of only NC organizations
nc_organizations_df = pf.organizations(state = 'NC', return_df=True)


#CLean DataFrame
to_drop = ['email', 'phone', 'url', 'photos', 'distance', 'organization_id','address.address2','address.country']
nc_organizations_df.drop(to_drop, inplace = True, axis=1)


#Rename Columns
new_columns = {
    'name': 'Name',
    'website': 'Website',
    'mission_statement':'Misson Statement',
    'address.address1': 'Address',
    'address.city':'City',
    'address.state':'State',
    'address.postcode':'Zip',
    'hours.friday':'Friday',
    'hours.saturday':'Saturday',
    'hours.sunday':'Sunday',
    'adoption.policy':'Adoption Policy',
    'adoption.url':'Adoption URL',
    'social_media.facebook':'Facebook',
    'social_media.twitter':'Twitter',
    'social_media.youtube':'YouTube',
    'social_media.instagram':'Instagram',
    'social_media.pinterest':'Pinterest'
}
nc_organizations_df.rename(columns=new_columns, inplace=True)
nc_organizations_df = nc_organizations_df.set_index('id')

#Data Frame of Organizations in NC
nc_organizations_df.head()

,Name,Website,Misson Statement,Address,City,State,Zip,hours.monday,hours.tuesday,hours.wednesday,...,Friday,Saturday,Sunday,Adoption Policy,Adoption URL,Facebook,Twitter,YouTube,Instagram,Pinterest
id,,,,,,,,,,,,,,,,,,,,,
NC1113,Angela's Ark,http://www.angelasark.org,Angela&#039;s Ark is a 501c3 animal rescue on ...,None,Charlotte,NC,28210,None,None,None,...,None,None,None,Thank you for your interest in our adoptable p...,http://www.angelasark.org,None,None,None,None,None
NC1037,Yahweh's Heart,None,Yahweh&#039;s Heart is perfect example of how ...,3442 Harvey Rd,Washington,NC,27889,8:30-3:30,8:30-3:30,8:30-3:30,...,8:30-3:30,By appointment only,By appointment only,None,None,https://www.facebook.com/Yahwehsheart/,None,None,None,None
NC1102,Snow Camp Sanctuary,https://snowcampfurever.com,"Our mission is to give a home to ferals, injur...",None,Snow Camp,NC,27349,None,None,None,...,None,None,None,None,None,https://www.facebook.com/SnowCamp2/,None,None,None,None
NC1070,Adoptions Ever After,None,None,None,Pineville,NC,28134,None,None,None,...,None,None,None,None,None,None,None,None,None,None
NC1079,Freedom Unchained inc.,None,None,None,Turkey,NC,28393,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [50]:
#Daily Schedule of organizations
'''towns_df = pd.DataFrame(university_towns,
...                         columns=['State', 'RegionName'])'''
schedule_df = pd.DataFrame(nc_organizations_df, columns = ['Friday', 'Saturday', 'Sunday'])
schedule_df

,Friday,Saturday,Sunday
id,,,
NC1113,None,None,None
NC1037,8:30-3:30,By appointment only,By appointment only
NC1102,None,None,None
NC1070,None,None,None
NC1079,None,None,None
NC1051,None,None,None
NC1093,None,None,None
NC1134,None,None,None
NC1138,None,None,None
